#### Madhur Jaripatke
#### Roll No. 50
#### BE A Computer
#### RMDSSOE, Warje, Pune

# Boston Housing Price Prediction using Deep Neural Networks

This notebook implements a deep neural network to predict housing prices using the Boston Housing dataset.

## Problem Statement:
**Linear regression by using Deep Neural network:** Implement Boston housing price prediction problem by Linear regression using Deep Neural network. Use Boston House price prediction dataset.

## Dataset Description

The Boston Housing dataset contains information about various features of houses in Boston suburbs and their prices. Each record has 13 features:

| Feature | Description |
|---------|-------------|
| CRIM | Per capita crime rate by town |
| ZN | Proportion of residential land zoned for lots over 25,000 sq.ft |
| INDUS | Proportion of non-retail business acres per town |
| CHAS | Charles River dummy variable (1 if tract bounds river; 0 otherwise) |
| NOX | Nitric oxides concentration (parts per 10 million) |
| RM | Average number of rooms per dwelling |
| AGE | Proportion of owner-occupied units built prior to 1940 |
| DIS | Weighted distances to five Boston employment centers |
| RAD | Index of accessibility to radial highways |
| TAX | Full-value property-tax rate per $10,000 |
| PTRATIO | Pupil-teacher ratio by town |
| B | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town |
| LSTAT | % lower status of the population |

**Target Variable**: MEDV (Median value of owner-occupied homes in $1000s)

## Import Required Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Data Loading and Preprocessing

We'll load the Boston Housing dataset and preprocess it by:
1. Loading from the original source
2. Normalizing features using StandardScaler
3. Splitting into train, validation, and test sets

In [ ]:
def load_boston_housing_data():
    """Load and preprocess the Boston Housing dataset from its original source."""
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]

    # Normalize features and target
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X = scaler_X.fit_transform(data)
    y = scaler_y.fit_transform(target.reshape(-1, 1))
    
    return X, y, scaler_y

# Load and preprocess data
X, y, scaler_y = load_boston_housing_data()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

## Model Architecture

We'll create a deep neural network with the following features:
- Batch normalization for training stability
- Dropout layers for regularization
- L2 regularization on dense layers
- Multiple hidden layers with decreasing units

In [ ]:
def create_model():
    """Creates an improved neural network model for housing price prediction."""
    model = tf.keras.Sequential([
        # Input layer
        tf.keras.layers.Input(shape=(13,)),
        tf.keras.layers.BatchNormalization(),
        
        # First hidden layer
        tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.3),
        
        # Second hidden layer
        tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),
        
        # Third hidden layer
        tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.1),
        
        # Output layer
        tf.keras.layers.Dense(1)
    ])
    
    return model

# Create and compile model
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

model.summary()

## Model Training

We'll train the model with:
- Early stopping to prevent overfitting
- Learning rate reduction on plateau
- Batch size of 32
- Maximum 200 epochs

In [ ]:
# Create TensorFlow datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=10,
        min_lr=0.00001,
        verbose=1
    )
]

# Train the model
history = model.fit(
    train_dataset,
    epochs=200,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

## Training History Visualization

Let's visualize how the model performed during training:

In [ ]:
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# MAE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE Over Time')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Model Evaluation and Predictions

Let's evaluate the model on the test set and make some sample predictions:

In [ ]:
# Evaluate on test set
test_results = model.evaluate(test_dataset, verbose=0)
print(f"Test Mean Absolute Error (normalized): {test_results[1]:.4f}")

# Make predictions
test_predictions = model.predict(X_test)

# Convert predictions back to original scale
test_predictions_original = scaler_y.inverse_transform(test_predictions)
test_actual_original = scaler_y.inverse_transform(y_test)

# Calculate metrics in original scale
mae_original = np.mean(np.abs(test_predictions_original - test_actual_original))
mse = np.mean((test_predictions_original - test_actual_original) ** 2)
rmse = np.sqrt(mse)
r2 = 1 - (np.sum((test_actual_original - test_predictions_original) ** 2) / 
          np.sum((test_actual_original - np.mean(test_actual_original)) ** 2))

print(f"\nTest Mean Absolute Error: ${mae_original:.2f}k")
print(f"Root Mean Square Error: ${rmse:.2f}k")
print(f"R-squared Score: {r2:.4f}")

# Show sample predictions
print("\nSample Predictions vs Actual Values (in $1000s):")
for pred, actual in zip(test_predictions_original[:5], test_actual_original[:5]):
    print(f"Predicted: ${pred[0]:.2f}k, Actual: ${actual[0]:.2f}k")

## Prediction vs Actual Plot

Let's visualize how well our predictions match the actual values:

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(test_actual_original, test_predictions_original, alpha=0.5)
plt.plot([test_actual_original.min(), test_actual_original.max()],
         [test_actual_original.min(), test_actual_original.max()],
         'r--', lw=2)
plt.xlabel('Actual Price ($1000s)')
plt.ylabel('Predicted Price ($1000s)')
plt.title('Predicted vs Actual House Prices')
plt.grid(True)
plt.show()